# 單位 Function
---
![單位 Function](https://raw.githubusercontent.com/Darkgrouptw/Machine-Learning-CS5087701/master/Identity%20function/imgs/Identity%20Function.png)
根據輸入的位置  
用 Hidden unit 壓縮成數字編碼  
在輸出

# Import lib
---

In [1]:
import tensorflow as tf
import numpy as np
import random

# 建構網路
---

In [2]:
# 要輸入幾個
vectorSize = 8

# 壓縮二進制
hiddenUnits = 2

In [3]:
# 輸入
inputValue = tf.placeholder(
    tf.float32,
    [None, vectorSize],
    name="InputValue"
)

# label 後的值
labelValue = tf.placeholder(
    tf.float32,
    [None, vectorSize],
    name="LabelValue"
)

In [4]:
weightInit = tf.random_normal_initializer(mean=0, stddev=0.3)
biasInit = tf.random_normal_initializer(mean=0, stddev=0.1)

In [5]:
hiddenlayer = tf.layers.dense(
    inputs=inputValue,
    units=hiddenUnits,
    activation=tf.nn.sigmoid,
    kernel_initializer=weightInit,
    bias_initializer= biasInit,
    name="HiddenLayer"
)

output = tf.layers.dense(
    inputs=hiddenlayer,
    units=vectorSize,
    kernel_initializer=weightInit,
    bias_initializer=biasInit,
    name="Output"
)

outputProb = tf.nn.softmax(output, name="OutputProb")

predValue = tf.argmax(outputProb, axis=1)

In [6]:
with tf.name_scope("Trainning"):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=labelValue)
    lossMean = tf.reduce_mean(loss)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-1).minimize(lossMean)

In [7]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [8]:
# logFile = tf.summary.FileWriter("./logs")
# logFile.add_graph(session.graph)

## 建 Training Data
---

In [9]:
# 所有可能的 DataSet
dataset = [
    [1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1]
            ]

In [10]:
# 測試資料的筆數
trainNum = 10000

In [11]:
# 產生 DataIndex & 產生 Train Set (trainNum)
trainSet = []
for i in range(0, trainNum):
    chooseIndex = random.randint(0, len(dataset) - 1)
    
    # 將資料
    trainSet.append(dataset[chooseIndex])

## Session 建構並開始學習
---

In [12]:
def printPred():
    for i in range(0, vectorSize):
        vInput = np.argmax(dataset[i])
        vHidden = session.run(hiddenlayer, feed_dict={inputValue: [dataset[i]]})
        vOutput = session.run(predValue, feed_dict={inputValue: [dataset[i]]})[0]
        print(format(vInput) + " => " + format(vHidden) + " => " + format(vOutput))

In [13]:
printPred()

0 => [[ 0.45042545  0.53688753]] => 5
1 => [[ 0.6218695   0.68029255]] => 5
2 => [[ 0.54299617  0.51187128]] => 5
3 => [[ 0.4703379   0.54338121]] => 5
4 => [[ 0.61967796  0.57213509]] => 5
5 => [[ 0.54344714  0.4211821 ]] => 5
6 => [[ 0.46185023  0.57694   ]] => 5
7 => [[ 0.40699914  0.59104496]] => 5


In [14]:
for i in range(0, trainNum, 10):
    feedDict = {inputValue: trainSet[i: i+10], labelValue: trainSet[i: i+10]}
    session.run(optimizer, feed_dict=feedDict)

## 看結果確率
---

In [15]:
printPred()

0 => [[ 0.3927485   0.99909687]] => 0
1 => [[  1.57824645e-04   9.99453843e-01]] => 1
2 => [[ 0.3652977   0.00075005]] => 2
3 => [[ 0.00037674  0.00016524]] => 3
4 => [[ 0.99945778  0.41294545]] => 4
5 => [[ 0.9987759  0.9994598]] => 5
6 => [[  9.99606073e-01   2.38707318e-04]] => 6
7 => [[ 0.03329074  0.34260309]] => 7


## 查看 Weight
---

In [16]:
hiddenWeight = tf.get_default_graph().get_tensor_by_name("HiddenLayer/kernel:0")
hiddenBias = tf.get_default_graph().get_tensor_by_name("HiddenLayer/bias:0")
print(session.run(hiddenWeight))
print("\n")
print(session.run(hiddenBias))

[[-0.85087222  6.89189339]
 [-9.16896725  7.39537096]
 [-0.96754205 -7.31139565]
 [-8.29867458 -8.8247509 ]
 [ 7.10410309 -0.46858165]
 [ 6.28924131  7.40621948]
 [ 7.42391014 -8.45681286]
 [-3.78371716 -0.76849473]]


[ 0.4150987   0.11677945]


In [17]:
outputWeight = tf.get_default_graph().get_tensor_by_name("Output/kernel:0")
outputBias = tf.get_default_graph().get_tensor_by_name("Output/kernel:0")
print(session.run(outputWeight))
print("\n")
print(session.run(outputBias))

[[ -6.15844965 -26.53968239  -1.52608299 -25.46878242   7.08772421
    5.18017244  12.09982967 -17.59918785]
 [  7.30821276  11.96138954 -18.46502876 -25.97653961  -5.43988323
    5.56522894 -23.837183    -1.22657943]]


[[ -6.15844965 -26.53968239  -1.52608299 -25.46878242   7.08772421
    5.18017244  12.09982967 -17.59918785]
 [  7.30821276  11.96138954 -18.46502876 -25.97653961  -5.43988323
    5.56522894 -23.837183    -1.22657943]]


# 結論
---
這個問題可能藉由更低的數字來逼近解  
因為在 hidden layer 的值  
可能也會成為分類的根據